In [4]:
import pandas as pd
import investpy
from forex_python.converter import CurrencyRates
import datetime
import plotly.express as px

def total_cost_eur(x):
    if x['currency'] != 'EUR':
        x['total_cost_eur'] = x['number_of_stocks'] * x['cost_per_stock_eur']
    else: 
        x['total_cost_eur'] = x['number_of_stocks'] * x['cost_per_stock_eur']
    return x

def shrink_pf(pf):
    pf = pf.sort_values('date')
    originals = pf[~pf.ticker.duplicated()].set_index('ticker')
    cols_to_add = ['number_of_stocks', 'total_cost_eur', 'transaction_costs']

    for i, row in pf[pf.ticker.duplicated()].iterrows():
        tick = row['ticker']
        originals.loc[tick, cols_to_add] = originals.loc[tick, cols_to_add] + row[cols_to_add]
    
    return originals.reset_index()

def exclude_weekends():
    today = datetime.datetime.now()
    if today.weekday() == 6:
        return today - datetime.timedelta(days=2)
    if today.weekday() == 0:
        return today - datetime.timedelta(days=3)
    return today - datetime.timedelta(days=1)

def get_hist_prices(pf, days_back=0):
    print('Retrieving prices...')

    # if days_back == 0:
    #     from_date = pf['date'].min().strftime('%d/%m/%Y')
    # elif days_back > len(_prices):
    #     print("More back than existing prices")
    #     from_date = (exclude_weekends() - datetime.timedelta(days=days_back)).strftime('%d/%m/%Y')
    # else: 
    #     print("prices already exist")
    #     return _prices.iloc[-days_back:]
        # from_date = _prices.iloc[-1].Date.strftime('%d/%m/%Y')

    to_date = exclude_weekends().date().strftime('%d/%m/%Y')  #(exclude_weekends() - datetime.timedelta(days=1))
    hist_prices = []
    for i, row in pf.iterrows():
        search_results = investpy.search_quotes(text=row['ticker'], products=[row['product']], countries=[row['country']])
        from_date = row['date'].strftime('%d/%m/%Y')
        for sr in search_results[:1]:
            hist_p = sr.retrieve_historical_data(from_date=from_date, to_date=to_date)['Close'].to_frame().rename({'Close':row['ticker']}, axis=1).interpolate()
            hist_prices.append(hist_p)
    
    return pd.concat(hist_prices, axis=1)

In [63]:
path = './data/'
portfolio = pd.read_excel(path + 'investing_source.xlsx', sheet_name='Stocks')
portfolio['date'] = portfolio['date'].dt.date
portfolio = portfolio.apply(total_cost_eur, axis=1).sort_values('total_cost_eur', ascending=False)

pf_no_dupl = shrink_pf(portfolio)
_prices = get_hist_prices(pf_no_dupl)


Retrieving prices...


In [5]:
l = [np.nan, 1, 2, 3, 4]

In [6]:
df = pd.DataFrame(l)

In [43]:
df.iloc[0:].first_valid_index()

1

In [57]:
portfolio[portfolio.ticker == "VWRL"]['number_of_stocks']

2    10
3    10
Name: number_of_stocks, dtype: int64

In [82]:
_prices.index.date >= portfolio['date'].iloc[0]

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [98]:
_prices['total'] = 0
_prices = _prices.fillna(0)
for i, row in portfolio.iterrows():
    _prices['total'] += _prices[row.ticker][_prices.index.date > row.date].mul(row.number_of_stocks)
    # print(_prices[row.ticker][_prices.index.date > row.date].mul(row.number_of_stocks))
    
_prices

,NVDA,BFIT,VWRL,total
Date,,,,
2020-07-06,393.57,0.00,0.00,NaN
2020-07-07,394.87,0.00,0.00,NaN
2020-07-08,408.64,0.00,0.00,NaN
2020-07-09,420.36,0.00,0.00,NaN
2020-07-10,419.17,0.00,0.00,NaN
...,...,...,...,...
2020-12-16,529.70,30.75,86.32,2717.35
2020-12-17,533.65,30.89,86.04,2717.80
2020-12-18,530.88,30.89,85.86,2711.43


In [84]:
_prices[_prices.index.date > row.date]

,NVDA,BFIT,VWRL,total
Date,,,,
2020-09-14,514.89,24.16,79.88,0
2020-09-15,519.64,24.05,80.56,0
2020-09-16,500.58,23.63,80.86,0
2020-09-17,498.54,23.36,79.90,0
2020-09-18,487.57,22.95,79.21,0
...,...,...,...,...
2020-12-16,529.70,30.75,86.32,0
2020-12-17,533.65,30.89,86.04,0
2020-12-18,530.88,30.89,85.86,0


In [56]:
_prices.apply(lambda x: x.mul(portfolio[portfolio.ticker == x.name]['number_of_stocks']), axis=1)

,NVDA,BFIT,VWRL
Date,,,
2020-07-06,NaN,NaN,NaN
2020-07-07,NaN,NaN,NaN
2020-07-08,NaN,NaN,NaN
2020-07-09,NaN,NaN,NaN
2020-07-10,NaN,NaN,NaN
...,...,...,...
2020-12-16,NaN,NaN,NaN
2020-12-17,NaN,NaN,NaN
2020-12-18,NaN,NaN,NaN


In [46]:
portfolio

,date,company,ticker,number_of_stocks,cost_per_stock,cost_per_stock_eur,currency,transaction_costs,color,product,country,sector,total_cost_eur
2,2020-09-11,Vanguard FTSE AW,VWRL,10,79.0,79.00,EUR,0.00,#4178B9,etfs,Netherlands,NaN,790.00
3,2020-10-28,Vanguard FTSE AW,VWRL,10,78.7,78.70,EUR,0.00,#4178B9,etfs,Netherlands,NaN,787.00
1,2020-08-27,Basic Fit,BFIT,15,23.6,23.60,EUR,2.11,#ff9900,stocks,Netherlands,NaN,354.00
0,2020-07-06,NVIDIA,NVDA,1,391.0,343.32,USD,0.50,#03883B,stocks,United States,Tech,343.32


In [45]:
_prices

,NVDA,BFIT,VWRL
Date,,,
2020-07-06,393.57,NaN,NaN
2020-07-07,394.87,NaN,NaN
2020-07-08,408.64,NaN,NaN
2020-07-09,420.36,NaN,NaN
2020-07-10,419.17,NaN,NaN
...,...,...,...
2020-12-16,529.70,30.75,86.32
2020-12-17,533.65,30.89,86.04
2020-12-18,530.88,30.89,85.86
